# 化合物表示学习和性质预测

在这篇教程中，我们将介绍如何运用图神经网络（GNN）模型来预测化合物的性质。具体来说，我们将演示如何对其进行预训练（pretrain），如何针对下游任务进行模型微调（finetune），并利用最终的模型进行推断（inference）。如果你想了解更多细节，请查阅 "[info graph](https://github.com/PaddlePaddle/PaddleHelix/apps/pretrained_compound/info_graph/README_cn.md)" 和 "[pretrained GNN](https://github.com/PaddlePaddle/PaddleHelix/apps/pretrained_compound/pretrain_gnns/README_cn.md)" 的详细解释.

# 第一部分：预训练

在这一部分，我们将展示如何预训练一个化合物 GNN 模型。本文中的预训练技术是在预训练 GNN 的基础上发展起来的，包括 attribute masking、context prediction 和有监督预训练。
更多细节请查看文件：`pretrain_attrmask.py`和 `pretrain_supervised.py`。

In [1]:
import os
import numpy as np
import sys
sys.path.insert(0, os.getcwd() + "/..")
os.chdir("../apps/pretrained_compound/pretrain_gnns")

PaddleHelix 是构建于 PaddlePaddle 之上的生物计算深度学习框架。

In [2]:
import paddle
import paddle.nn as nn
import paddle.distributed as dist
import pgl

from pahelix.model_zoo.pretrain_gnns_model import PretrainGNNModel, AttrmaskModel
from pahelix.datasets.zinc_dataset import load_zinc_dataset
from pahelix.utils.splitters import RandomSplitter
from pahelix.featurizers.pretrain_gnn_featurizer import AttrmaskTransformFn, AttrmaskCollateFn
from pahelix.utils import load_json_config

2021-05-08 15:31:52,712 - INFO - ujson not install, fail back to use json instead
2021-05-08 15:31:52,801 - INFO - Enabling RDKit 2020.09.1 jupyter extensions


## 加载配置

这里，我们使用 `compound_encoder_config`和`model_config` 保存模型配置。`PretrainGNNModel`是用于预训练gnns的基本GNN模型，`AttrmaskModel` 是一种无监督的预训练模型，它随机地对某个节点的原子类型进行 mask，然后再尝试去预测这个原子的类型。同时，我们使用 Adam 优化器并将学习率（learning rate）设置为 0.001。


In [3]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/pre_Attrmask.json")

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = AttrmaskModel(model_config, compound_encoder)
opt = paddle.optimizer.Adam(0.001, parameters=model.parameters())

/home/bio/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[PretrainGNNModel] embed_dim:300
[PretrainGNNModel] dropout_rate:0.5
[PretrainGNNModel] norm_type:batch_norm
[PretrainGNNModel] graph_norm:False
[PretrainGNNModel] residual:False
[PretrainGNNModel] layer_num:5
[PretrainGNNModel] gnn_type:gin
[PretrainGNNModel] JK:last
[PretrainGNNModel] readout:mean
[PretrainGNNModel] atom_names:['atomic_num', 'chiral_tag']
[PretrainGNNModel] bond_names:['bond_dir', 'bond_type']


## 数据集加载和特征提取
###用`wget`下载数据集
我们首先使用 `wget` 来下载一个小型的测试数据集，如果你的本地计算机上没有 `wget`，你也可以复制下面的链接到你的浏览器中来下载数据。但是请注意你需要把数据包移动到这个路径："../apps/pretrained_compound/pretrain_gnns/"。

In [4]:
### Download a toy dataset for demonstration:
!wget "https://baidu-nlp.bj.bcebos.com/PaddleHelix%2Fdatasets%2Fcompound_datasets%2Fchem_dataset_small.tgz" --no-check-certificate
!tar -zxf "PaddleHelix%2Fdatasets%2Fcompound_datasets%2Fchem_dataset_small.tgz"
!ls "./chem_dataset_small"
### Download the full dataset as you want:
# !wget "http://snap.stanford.edu/gnn-pretrain/data/chem_dataset.zip" --no-check-certificate
# !unzip "chem_dataset.zip"
# !ls "./chem_dataset"

--2021-05-08 15:32:00--  https://baidu-nlp.bj.bcebos.com/PaddleHelix%2Fdatasets%2Fcompound_datasets%2Fchem_dataset_small.tgz
Resolving baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)... 10.70.0.165
Connecting to baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 609563 (595K) [application/gzip]
Saving to: ‘PaddleHelix%2Fdatasets%2Fcompound_datasets%2Fchem_dataset_small.tgz.2’

100%[======================================>] 609,563     --.-K/s   in 0.04s   

2021-05-08 15:32:01 (14.0 MB/s) - ‘PaddleHelix%2Fdatasets%2Fcompound_datasets%2Fchem_dataset_small.tgz.2’ saved [609563/609563]

tox21  zinc_standard_agent


### 加载数据集并生成特征
这里我们采用 Zinc 数据集来进行预训练。这里我们用的是小数据集用作解释，你可以加载全部的数据集。
使用 `AttrmaskTransformFn` 来配合模型 `AttrmaskModel`。它用于生成特征，原始特征被处理为网络上可用的特征，例如，smiles变成节点和边特征。

In [5]:
### Load the first 1000 of the toy dataset for speed up
dataset = load_zinc_dataset("./chem_dataset_small/zinc_standard_agent/")
dataset = dataset[:1000]
print("dataset num: %s" % (len(dataset)))

transform_fn = AttrmaskTransformFn()
dataset.transform(transform_fn, num_workers=2)

dataset num: 1000


## 启动训练

现在我们开始训练 Attrmask 模型。我们仅训练两个 epoch 作为演示，在这里，我们使用`AttrmaskTransformFn`将多个样本聚合到一个mini-batch中。数据加载的过程通过4个 `workers` 进行了加速。然后我们将预训练后的模型保存到 "./model/pretrain_attrmask"，作为下游任务的初始模型。

In [6]:
def train(model, dataset, collate_fn, opt):
    data_gen = dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=True,
            collate_fn=collate_fn)
    list_loss = []
    model.train()
    for graphs, masked_node_indice, masked_node_label in data_gen:
        graphs = graphs.tensor()
        masked_node_indice = paddle.to_tensor(masked_node_indice, 'int64')
        masked_node_label = paddle.to_tensor(masked_node_label, 'int64')
        loss = model(graphs, masked_node_indice, masked_node_label)
        loss.backward()
        opt.step()
        opt.clear_grad()
        list_loss.append(loss.numpy())
    return np.mean(list_loss)

collate_fn = AttrmaskCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'],
        mask_ratio=0.15)

for epoch_id in range(2):
    train_loss = train(model, dataset, collate_fn, opt)
    print("epoch:%d train/loss:%s" % (epoch_id, train_loss))
paddle.save(compound_encoder.state_dict(), 
        './model/pretrain_attrmask/compound_encoder.pdparams')

/home/bio/tools/paddle2.0/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


epoch:0 train/loss:2.835863
epoch:1 train/loss:0.9871801


模型预训练的内容到此为止，你可以根据自己的需要对上面的参数进行调整。

# 第二部分：下游任务模型微调（fintune）

下面我们将介绍如何对预训练的模型进行微调来适应下游任务。

更多细节参见 `finetune.py` 文件中的内容。

In [7]:
from pahelix.utils.splitters import \
    RandomSplitter, IndexSplitter, ScaffoldSplitter
from pahelix.datasets import *

from src.model import DownstreamModel
from src.featurizer import DownstreamTransformFn, DownstreamCollateFn
from src.utils import calc_rocauc_score, exempt_parameters

下游任务的数据集通常规模很小，并且面向不同的任务。例如，BBBP 数据集用于预测化合物的血脑屏障通透性；Tox21 数据集用于预测化合物的毒性等。这里我们使用 Tox21 数据集进行演示。

In [8]:
task_names = get_default_tox21_task_names()
print(task_names)

['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']


## 加载配置

我们使用 `compound_encoder_config` 和 `model_config` 来加载模型配置，注意这里的模型结构的设置应该和预训练模型中的设置保持一致，否则模型加载将会失败。
`DownstreamModel` 是一个有监督的 GNN 模型，用于上述 `task_names` 中定义的预测任务。

同时，我们以BCEloss为准则，使用Adam优化器将lr设置为0.001。

In [9]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/down_linear.json")
model_config['num_tasks'] = len(task_names)

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = DownstreamModel(model_config, compound_encoder)
criterion = nn.BCELoss(reduction='none')
opt = paddle.optimizer.Adam(0.001, parameters=model.parameters())

[PretrainGNNModel] embed_dim:300
[PretrainGNNModel] dropout_rate:0.5
[PretrainGNNModel] norm_type:batch_norm
[PretrainGNNModel] graph_norm:False
[PretrainGNNModel] residual:False
[PretrainGNNModel] layer_num:5
[PretrainGNNModel] gnn_type:gin
[PretrainGNNModel] JK:last
[PretrainGNNModel] readout:mean
[PretrainGNNModel] atom_names:['atomic_num', 'chiral_tag']
[PretrainGNNModel] bond_names:['bond_dir', 'bond_type']


## 加载预训练模型

加载预训练阶段得到的模型。这里我们加载模型 "pretrain_attrmask" 作为一个例子。

In [10]:
compound_encoder.set_state_dict(paddle.load('./model/pretrain_attrmask/compound_encoder.pdparams'))

## 数据加载和特征提取

将 `DownstreamTransformFn` 与 `DownstreamModel` 一起使用。它用于生成特征，原始特征被处理为网络上可用的特征，例如，smiles字符串变成节点和边特征。

Tox21 数据集用作下游任务数据集，我们使用 `ScaffoldSplitter` 将数据集拆分为训练/验证/测试集。`ScaffoldSplitter` 首先根据 Bemis-Murcko scaffold 对化合物进行排序，然后从前到后，将参数 `frac_train` 定义的比例的数据作为训练集，将 `frac_valid` 定义的比例的数据作为验证集，其余的作为测试集。`ScaffoldSplitter` 能更好地评价模型对非同分布样本的泛化能力。这里也可以使用其他的拆分器，如 `RandomSplitter`、`RandomScaffoldSplitter` 和 `IndexSplitter`。

In [11]:
### Load the toy dataset:
dataset = load_tox21_dataset("./chem_dataset_small/tox21", task_names)
### Load the full dataset:
# dataset = load_tox21_dataset("./chem_dataset/tox21", task_names)
dataset.transform(DownstreamTransformFn(), num_workers=4)

# splitter = RandomSplitter()
splitter = ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.split(
        dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1)
print("Train/Valid/Test num: %s/%s/%s" % (
        len(train_dataset), len(valid_dataset), len(test_dataset)))

RDKit WARNING: [15:32:26] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [15:32:36] WARNING: not removing hydrogen atom without neighbors


Train/Valid/Test num: 6264/783/784


## 启动训练

出于演示的目的，这里我们只将 attrmask 模型训练了4轮。在这里，我们使用`DownstreamCollateFn`将多个样本聚合到一个mini-batch中。由于每个下游任务都包含了多个子任务，我们分别计算了每个子任务的 roc-auc，在求其均值作为最后的评估标准。

In [12]:
def train(model, train_dataset, collate_fn, criterion, opt):
    data_gen = train_dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=True,
            collate_fn=collate_fn)
    list_loss = []
    model.train()
    for graphs, valids, labels in data_gen:
        graphs = graphs.tensor()
        labels = paddle.to_tensor(labels, 'float32')
        valids = paddle.to_tensor(valids, 'float32')
        preds = model(graphs)
        loss = criterion(preds, labels)
        loss = paddle.sum(loss * valids) / paddle.sum(valids)
        loss.backward()
        opt.step()
        opt.clear_grad()
        list_loss.append(loss.numpy())
    return np.mean(list_loss)

def evaluate(model, test_dataset, collate_fn):
    data_gen = test_dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=False,
            collate_fn=collate_fn)
    total_pred = []
    total_label = []
    total_valid = []
    model.eval()
    for graphs, valids, labels in data_gen:
        graphs = graphs.tensor()
        labels = paddle.to_tensor(labels, 'float32')
        valids = paddle.to_tensor(valids, 'float32')
        preds = model(graphs)
        total_pred.append(preds.numpy())
        total_valid.append(valids.numpy())
        total_label.append(labels.numpy())
    total_pred = np.concatenate(total_pred, 0)
    total_label = np.concatenate(total_label, 0)
    total_valid = np.concatenate(total_valid, 0)
    return calc_rocauc_score(total_label, total_pred, total_valid)

collate_fn = DownstreamCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'])
for epoch_id in range(4):
    train_loss = train(model, train_dataset, collate_fn, criterion, opt)
    val_auc = evaluate(model, valid_dataset, collate_fn)
    test_auc = evaluate(model, test_dataset, collate_fn)
    print("epoch:%s train/loss:%s" % (epoch_id, train_loss))
    print("epoch:%s val/auc:%s" % (epoch_id, val_auc))
    print("epoch:%s test/auc:%s" % (epoch_id, test_auc))
paddle.save(model.state_dict(), './model/tox21/model.pdparams')

Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:0 train/loss:0.26803324
epoch:0 val/auc:0.6266238165767454
epoch:0 test/auc:0.6216414776039183


/home/bio/tools/paddle2.0/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:1 train/loss:0.22468299
epoch:1 val/auc:0.6659571793610889
epoch:1 test/auc:0.6539646927043956


/home/bio/tools/paddle2.0/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:2 train/loss:0.21930875
epoch:2 val/auc:0.6822124992478676
epoch:2 test/auc:0.6736089418895174


/home/bio/tools/paddle2.0/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:3 train/loss:0.21560585
epoch:3 val/auc:0.6715751869311037
epoch:3 test/auc:0.6290005037852295


# 第三部分：下游任务模型预测
在这部分，我们将简单介绍如何利用训好的下游任务模型来对给定的 SMILES 序列做预测。

## 加载配置
这部分跟第二部分的基本相同。

In [13]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/down_linear.json")
model_config['num_tasks'] = len(task_names)

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = DownstreamModel(model_config, compound_encoder)

[PretrainGNNModel] embed_dim:300
[PretrainGNNModel] dropout_rate:0.5
[PretrainGNNModel] norm_type:batch_norm
[PretrainGNNModel] graph_norm:False
[PretrainGNNModel] residual:False
[PretrainGNNModel] layer_num:5
[PretrainGNNModel] gnn_type:gin
[PretrainGNNModel] JK:last
[PretrainGNNModel] readout:mean
[PretrainGNNModel] atom_names:['atomic_num', 'chiral_tag']
[PretrainGNNModel] bond_names:['bond_dir', 'bond_type']


/home/bio/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 加载训练好的下游任务模型
加载在第二部分中训练好的下游任务模型。

In [15]:
model.set_state_dict(paddle.load('./model/tox21/model.pdparams'))

## 开始预测
对给定的 SMILES 序列进行预测。我们直接调用 `DownstreamTransformFn` 和 `DownstreamCollateFn` 函数将原始的 SMILES 序列转化为模型的输入。

以 Tox21 数据集为例，我们的下游任务模型可以给出 Tox21 里面的12个子任务的预测。

In [16]:
SMILES="O=C1c2ccccc2C(=O)C1c1ccc2cc(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c2n1"
transform_fn = DownstreamTransformFn(is_inference=True)
collate_fn = DownstreamCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'],
        is_inference=True)
graph = collate_fn([transform_fn({'smiles': SMILES})])
preds = model(graph.tensor()).numpy()[0]
print('SMILES:%s' % SMILES)
print('Predictions:')
for name, prob in zip(task_names, preds):
    print("  %s:\t%s" % (name, prob))

SMILES:O=C1c2ccccc2C(=O)C1c1ccc2cc(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c2n1
Predictions:
  NR-AR:	0.32046446
  NR-AR-LBD:	0.2179917
  NR-AhR:	0.43747446
  NR-Aromatase:	0.3693441
  NR-ER:	0.38625807
  NR-ER-LBD:	0.26430473
  NR-PPAR-gamma:	0.31611276
  SR-ARE:	0.436297
  SR-ATAD5:	0.28199005
  SR-HSE:	0.24803819
  SR-MMP:	0.42311215
  SR-p53:	0.22394522


/home/bio/tools/paddle2.0/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
